<a href="https://colab.research.google.com/github/ayeshasGithub/hello-world/blob/master/Feature__Engineering_start_with.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/LMK_WoolSales_SaleSummary.csv', encoding= 'unicode_escape')
df_big=df
df_small=df.sample(frac=0.01, random_state=1)#subset of df
#fig = X_train.groupby(['Cabin_ordered'])['Survived'].mean().plot() #group by
#df.shape #how many rows and columns
#df['Brand'].head(5)
#df.columns #see column names
#df.isna().sum() #which column has how many null values

#remove 3600 rows based on sold outcome
df_sold = df[df['SaleOutcome'] == 0] 

'''
#df_sold-df_test using sklearn easy technique
from sklearn.model_selection import train_test_split
df_sold_train, df_sold_test = train_test_split(df_sold, test_size=0.00001)
df_all_train, df_all_test = train_test_split(df, test_size=0.00001)

print(df_sold_train.shape)
print(df_sold_test.shape)
'''
print(df_sold.head(5))

!pip install pickle-mixin
import pickle


In [18]:
def extract_year_month_day(df_date,what):
  import datetime
  df_date = df_date.astype("datetime64")#convert type to datetime64
  if what=='day':
    df_date['Day']=df_date['SaleDate'].dt.day#extract month
  if what=='month':
    df_date['Month']=df_date['SaleDate'].dt.month#extract month
  if what=='year':
    df_date['Year']=df_date['SaleDate'].dt.year#extract year

  return df_date

  


def see_correlation(df_test,column_names,target_name,df_t):#df_test=> dataframe with column and target
  import numpy
  #df_test[target_name]=df[[target_name]]#add target column
  print('helo see_correlation')
  for col in column_names:
    encoded_feature = df_test[col].values
    #first_correlation = numpy.corrcoef(df_test[target_name].values, encoded_feature)[0][1]
    first_correlation = numpy.corrcoef(df_t[target_name].values, encoded_feature)[0][1]
    print(col,' correlation ', first_correlation)
        

#normalization function
def scaleColumns(df_test, cols_to_scale):
    #import libraries
    import pandas as pd
    from sklearn import preprocessing
    min_max_scaler = preprocessing.MinMaxScaler()
    print('cols to scale',cols_to_scale)
    for col in cols_to_scale:
      df_test[col] = pd.DataFrame(min_max_scaler.fit_transform(pd.DataFrame(df_test[col])),columns=[col])
    return df_test



#standardization function
def standardColumns(df, cols_to_scale):
    from sklearn.preprocessing import StandardScaler
    import pandas as pd
    scaler=StandardScaler()
    
    for col in cols_to_scale:
        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(df[col])),columns=[col])
    return df



def label_encoder_date(df_test,columns):
  from sklearn import preprocessing
  le = preprocessing.LabelEncoder()
  
  for col in columns:
    encoded=le.fit_transform(df_test[col].astype(str))
    df_test[col] = pd.DataFrame({col: encoded})
  #'''  
  return df_test



# https://necromuralist.github.io/kaggle-competitions/posts/mean-encoding-the-competition-data/
def meanEncoding(df_c, cols_to_scale,target,df_t):#df_t==> column with target values
    import numpy 
    df_c[target]=df_t[target]   #add target column to df_c
    for col in cols_to_scale:
        print(col,end=' ')
        item_id_target_mean = df_c.groupby([col])[target].mean()
        #item_id_target_mean = df_c.groupby([col])[target].transform('mean')
        df_c[col] = df_c[col].map(item_id_target_mean)
        encoded_feature = df_c[col].values
        first_correlation = numpy.corrcoef(df_c[target].values, encoded_feature)[0][1]
        #first_correlation = numpy.corrcoef(df_t[target].values, encoded_feature)[0][1]
        print(first_correlation)
    df_c=df_c.loc[:,cols_to_scale]    
    return df_c






# https://necromuralist.github.io/kaggle-competitions/posts/mean-encoding-the-competition-data/
def kfoldSchemeEncoding(df_c, cols_to_scale,target):
      
    folder = KFold(n_splits=5, shuffle=False)
    train_new = pd.DataFrame(index=df_c.index, columns=df_c.columns) 
    
    for col in cols_to_scale:
    
        column = col#"item_id"
        encoded_column = column + "_mean_target"
        
           #all_data=df_c
        train_new[encoded_column] = numpy.nan
        


        for training_index, validation_index in folder.split(df_c):

          #print('index',training_index,validation_index)
          #print(encoded_column)


          x_train = df_c.iloc[training_index].copy()
          x_validation = df_c.iloc[validation_index].copy()
          means = x_validation[column].map(x_train.groupby(column)[target].mean())
          x_validation[encoded_column] = means
          # train_new is a dataframe copy we made of the training data
          train_new.iloc[validation_index] = x_validation

        
        #train_new.fillna(NAN_VALUE, inplace=True)
        encoded_feature = train_new[encoded_column].values#.SaleNbrSellingCntr_mean_target.values#item_id_mean_target.values
        corr = numpy.corrcoef(df_c[target].values, encoded_feature)[0][1]
        print(corr)
        #grader.submit_tag('KFold_scheme', corr)
        #'''
def check_for_nulls(df_test):
  print(df_test.isna().sum())

In [19]:
def apply_label_encoder_date(df_test, columns,purpose='train'):#it needs to pass in the train data (df_tr) also during making test set since we are not saving anything
  from sklearn import preprocessing
  le = preprocessing.LabelEncoder()

  if purpose=='train':

    #save training data
    df_tr=df_test
    with open('train_data_date.pickle', 'wb') as f:
      pickle.dump(df_tr, f)
    
    for col in columns:
      encoded=le.fit_transform(df_test[col].astype(str))
      df_test[col] = pd.DataFrame({col: encoded})
        
    return df_test

  else:
    print('label on test data')
    #load df_tr
    with open('train_data_date.pickle', 'rb') as f:
      df_tr = pickle.load(f)
    print(df_tr.columns)
    print(df_test.columns)
    for col in columns:
      le.fit(df_tr[col].astype(str))
      encoded=le.transform(df_test[col].astype(str))
      df_test[col] = pd.DataFrame({col: encoded})

    return df_test

In [20]:
#https://stackoverflow.com/questions/21057621/sklearn-labelencoder-with-never-seen-before-values
#fit(self, y)-->Fit label encoder, le.fit(X)
#fit_transform(self, y)--> Fit label encoder and return encoded labels

def apply_label_encoder(df_test, columns,purpose='train'):#it needs to pass in the train data (df_tr) also during making test set since we are not saving anything
  from sklearn import preprocessing
  le = preprocessing.LabelEncoder()

  if purpose=='train':

    #save training data
    df_tr=df_test
    with open('train_data.pickle', 'wb') as f:
      pickle.dump(df_tr, f)
    
    for col in columns:
      encoded=le.fit_transform(df_test[col].astype(str))
      df_test[col] = pd.DataFrame({col: encoded})
        
    return df_test

  else:
    print('label on test data')
    #load df_tr
    with open('train_data.pickle', 'rb') as f:
      df_tr = pickle.load(f)
    print(df_tr.columns)
    print(df_test.columns)
    for col in columns:
      le.fit(df_tr[col].astype(str))
      encoded=le.transform(df_test[col].astype(str))
      df_test[col] = pd.DataFrame({col: encoded})

    return df_test

In [4]:
  
# https://necromuralist.github.io/kaggle-competitions/posts/mean-encoding-the-competition-data/
def apply_meanEncoding(df_c, cols_to_scale,target,df_t,purpose='train'):#df_t==> column with target values
    import numpy 
    item_id_target_mean_dic={}
    if purpose=='train':
      df_c[target]=df_t[target]   #add target column to df_c
      for col in cols_to_scale:
          print(col,end=' ')
          item_id_target_mean = df_c.groupby([col])[target].mean()
          item_id_target_mean_dic[col]=item_id_target_mean
          df_c[col] = df_c[col].map(item_id_target_mean)
          encoded_feature = df_c[col].values
          first_correlation = numpy.corrcoef(df_c[target].values, encoded_feature)[0][1]
          print(first_correlation)
      #save item_id_target_mean_dic
      with open('mean_encoded.pickle', 'wb') as f:
        pickle.dump(item_id_target_mean_dic, f)
      
      df_c=df_c.loc[:,cols_to_scale] 
      
      

    else:
        print('implement')
        #load what 
        with open('mean_encoded.pickle', 'rb') as f:
          item_id_target_mean_dic= pickle.load(f)

        for col in cols_to_scale:
          item_id_target_mean=item_id_target_mean_dic[col]
          df_c[col] = df_c[col].map(item_id_target_mean)
          
    return df_c


In [28]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold #KFold splits data set in the given number of folds

# This way we have randomness and are able to reproduce the behaviour within this cell.
np.random.seed(13)

def impact_coding(data, feature, target='SalePrice'):
    '''
    In this implementation we get the values and the dictionary as two different steps.
    This is just because initially we were ignoring the dictionary as a result variable.
   
    In this implementation the KFolds use shuffling. If you want reproducibility the cv could be moved to a parameter.
    '''
    n_folds = 10 # data will be splitted into 20 folds
    n_inner_folds = 5 #data will be splitted into 10 folds
    impact_coded = pd.Series()
    
    oof_default_mean = data[target].mean() # Gobal mean to use by default (you could further tune this)
    kf = KFold(n_splits=n_folds, shuffle=True) # shuffle data before splitting into batches. Note that the samples within each split will not be shuffled.
    oof_mean_cv = pd.DataFrame()
    split = 0
    county=0
    #train_index=infold, test_index=oof
    for infold, oof in kf.split(data[feature]):# the kf class's split, splits data into 20(given) folds and each fold is called by a loop

            impact_coded_cv = pd.Series()
            kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)#for one split, kf_inner again splits it to 10 folds
            inner_split = 0
            inner_oof_mean_cv = pd.DataFrame()
            oof_default_inner_mean = data.iloc[infold][target].mean()# 1st mean, not group by feature
            county=county+1
            print('inner',county)
            for infold_inner, oof_inner in kf_inner.split(data.iloc[infold]):
                # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)
                oof_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean() #2nd mean, group by feature
                
                impact_coded_cv = impact_coded_cv.append(data.iloc[infold].apply(
                            lambda x: oof_mean[x[feature]]
                                      if x[feature] in oof_mean.index
                                      else oof_default_inner_mean
                            , axis=1))

                # Also populate mapping (this has all group -> mean for all inner CV folds)
                inner_oof_mean_cv = inner_oof_mean_cv.join(pd.DataFrame(oof_mean), rsuffix=inner_split, how='outer')
                inner_oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True)
                inner_split += 1

            # Also populate mapping
            oof_mean_cv = oof_mean_cv.join(pd.DataFrame(inner_oof_mean_cv), rsuffix=split, how='outer')
            oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
            split += 1
            
            impact_coded = impact_coded.append(data.iloc[oof].apply(
                            lambda x: inner_oof_mean_cv.loc[x[feature]].mean()
                                      if x[feature] in inner_oof_mean_cv.index
                                      else oof_default_mean
                            , axis=1))

    return impact_coded, oof_mean_cv.mean(axis=1), oof_default_mean



# Apply the encoding to training and test data, and preserve the mapping
def apply_impact_encoder(df_c, cols_to_scale,target,df_t,purpose='train'):
  #categorical_features=features_c
  #train_data=df_big
  #add tareget column
  df_c[target]=df_t
  if purpose=='train':
    impact_coding_map_dic = {}
    for col in cols_to_scale:
        print("Impact coding for {}".format(col))
        df_c[col], impact_coding_mapping, default_coding = impact_coding(df_c, col)
        impact_coding_map_dic[col] = (impact_coding_mapping, default_coding)
    
    #save item_id_target_mean_dic
    with open('impact.pickle', 'wb') as f:
        pickle.dump(impact_coding_map_dic, f)
 

  else:
    
    with open('impact.pickle', 'rb') as f:
      impact_coding_map_dic = pickle.load(f)

    for col in cols_to_scale:
      
      mapping, default_mean = impact_coding_map_dic[col]

      print(col, 'here',mapping, default_mean)
      df_c[col] = df_c.apply(lambda x: mapping[x[col]]
                                            if x[col] in mapping 
                                            else default_mean
                                                               , axis=1)
  return df_c[cols_to_scale]#return only scaled ones without tagret
    

In [ ]:
#feature set 4 (with mean encoding and without not-sold wool, not sold wool are filtered)
def select_engg_features(df_test,purpose):#update this function based on selected feature, encoding you want to apply and filter technique
    '''
      * the categorical (seller) variables ==> label+impact
    '''
    df=df_test
    #target variable:
    label=['SalePrice']
    target=['SalePrice']
    df_target=df.loc[:,target]#create view


    #weight variables:
    features_w=['NetWeight']#from all the weights only using net weight since others are correlated
    df_w=df.loc[:,features_w]#create view
    df_w=scaleColumns(df_w,features_w)#apply normalization
    #see_correlation(df_w,features_w,'SalePrice',df_target)
    #check_for_nulls(df_w)


    #date variables:
    features_d=['SaleDate']#SaleDate will change to 'Month'
    df_date=df.loc[:,features_d]#create view
    df_date= extract_year_month_day(df_date,'month')#extract month column, this will be added to the dataframe as a new column 'Month'
    df_d=apply_label_encoder_date(df_date[['Month']],['Month'],purpose)#apply label encoder: deal month as categorical or the model would give the most importance to 12
    features_d=['Month']
    #see_correlation(df_d,features_d,'SalePrice',df_target)
    #check_for_nulls(df_d)

    #seller variables:
    features_c_1=['SaleNbrSellingCntr', 'Brand','StandardBaleDesc']
    features_c_2=['Region','Division','Branch','WAM','Agent']
    features_c=features_c_1+features_c_2
    df_c=df.loc[:,features_c]#create view
    df_c.fillna('unknown')#replace missing values
    df_c=apply_label_encoder(df_c,features_c,purpose)#apply label encoder
    #see_correlation(df_c,features_c,'SalePrice',df_target)
    #df_c=apply_meanEncoding(df_c, features_c,'SalePrice',df_target,purpose)#apply mean encoder after label encoder, mean encoding displays correlation also
    df_c=apply_impact_encoder(df_c, features_c,'SalePrice',df_target,purpose)

    #check_for_nulls(df_c)



    #quality variables: 
    features_q_1 = ['Micron','Curvature','VMB','Yield1','FolioProceeds']
    features_q_2=['FDMean','FDCOEfficient','FDComfort']
    features_q_3=['StapleLength','StapleLengthCV','StapleStrength','StapleStrengthLowest25']
    features_q_4=['TheoreticalHauteur','TheoreticalHauteurCV']
    features_q_5=['PositionofBreakTip','PositionofBreakMiddle','PositionofBreakBase']
    features_q=features_q_1+features_q_2+features_q_3+features_q_4+features_q_5
    df_q=df.loc[:,features_q]#create view
    df_q=standardColumns(df_q,features_q)#apply standardization technique
    #see_correlation(df_q,features_q,'SalePrice',df_target)
    #check_for_nulls(df_q)


    #Filter variables(sold or not sold?):
    features_f=['SaleOutcome']
    df_f=df.loc[:,features_f]


    #features you want
    features=features_w+features_d+features_c+features_q+target#you want these features (names only)

    #combine columns
    data=[df_w,df_d,df_c,df_q,df_target]#data = [df1["A"], df2["A"]]
    df_featuere_set = pd.concat(data, axis=1)
    print(df_featuere_set.head(5))

    #filter
    
    data=[df_w,df_d,df_c,df_q,df_f,df_target]#make data again with filter variable
    df_temp = pd.concat(data, axis=1)#, keys=headers)
    print(df_temp.columns)
    df_temp= df_temp[df_temp['SaleOutcome'] == 0] 
    print(df_temp.head(5))
    df_featuere_set = df_temp.loc[:,features]#dataframe of features without the filter variable
    #'''

    print(df_featuere_set.head(5))
    return df_featuere_set, features

#feature set 4



df_feature_set_4, feature_names=select_engg_features(df_big,'train')
with open('f4_label_impact_5_10.pickle', 'wb') as f:
  pickle.dump(df_feature_set_4, feature_names, f)
 

#df_feature_set_4_last_50, feature_names=select_engg_features(df_big.loc[-50:],'test')







cols to scale ['NetWeight']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Impact coding for SaleNbrSellingCntr
inner 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


inner 2
inner 3
inner 4
inner 5
inner 6
inner 7
inner 8
inner 9
inner 10
Impact coding for Brand
inner 1
inner 2
inner 3
inner 4
inner 5
inner 6
inner 7
inner 8
inner 9
inner 10
Impact coding for StandardBaleDesc
inner 1
inner 2
inner 3
inner 4
inner 5
inner 6
inner 7
inner 8
inner 9
inner 10
Impact coding for Region
inner 1
inner 2
inner 3
inner 4
inner 5
inner 6
inner 7
inner 8
inner 9
inner 10
Impact coding for Division
inner 1
inner 2
inner 3
inner 4
inner 5
inner 6
inner 7
inner 8
inner 9
inner 10
Impact coding for Branch
inner 1
inner 2
inner 3
inner 4
inner 5
inner 6
inner 7
inner 8
inner 9
inner 10
Impact coding for WAM
inner 1
inner 2
inner 3
inner 4
inner 5
inner 6
inner 7
inner 8
inner 9
inner 10
Impact coding for Agent
inner 1
inner 2
inner 3
inner 4
inner 5
inner 6
inner 7
inner 8
inner 9


In [24]:
print(df_feature_set_4.loc[-50:].head(5))
print(df_feature_set_4_last_50.head(5))

   NetWeight  Month  ...  PositionofBreakBase  SalePrice
0   0.101094      1  ...             1.652904     1250.0
1   0.065628      1  ...             0.418474     1198.0
2   0.086178      1  ...            -0.724517     1208.0
3   0.070103      1  ...            -0.724517     1208.0
4   0.047232      1  ...             0.281315     1411.0

[5 rows x 28 columns]
   NetWeight  Month  ...  PositionofBreakBase  SalePrice
0   0.101094      1  ...             1.652904     1250.0
1   0.065628      1  ...             0.418474     1198.0
2   0.086178      1  ...            -0.724517     1208.0
3   0.070103      1  ...            -0.724517     1208.0
4   0.047232      1  ...             0.281315     1411.0

[5 rows x 28 columns]


In [23]:
df_feature_set_4_last_50, feature_names=select_engg_features(df_big.loc[-50:],'test')

cols to scale ['NetWeight']
label on test data
Index(['Month'], dtype='object')
Index(['Month'], dtype='object')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


label on test data
Index(['SaleNbrSellingCntr', 'Brand', 'StandardBaleDesc', 'Region', 'Division',
       'Branch', 'WAM', 'Agent'],
      dtype='object')
Index(['SaleNbrSellingCntr', 'Brand', 'StandardBaleDesc', 'Region', 'Division',
       'Branch', 'WAM', 'Agent'],
      dtype='object')
SaleNbrSellingCntr here SaleNbrSellingCntr
0    973.418888
1    949.767007
2    982.615005
dtype: float64 833.1187716340875
Brand here Brand
0        1137.866667
1         861.683333
3         817.500000
4        1004.437500
5        1160.500000
            ...     
16053     774.650000
16054     867.453682
16055     739.093205
16056     900.453682
16057     755.979167
Length: 11245, dtype: float64 833.1187716340875
StandardBaleDesc here StandardBaleDesc
0        766.059385
1       1036.559385
2       1098.000000
3       1160.559385
4       1051.559385
           ...     
6235    1554.291667
6237    1653.866667
6238    1602.166667
6239     534.059385
6241     281.630729
Length: 3820, dtype: float64 8

In [ ]:
    #Train set experiment on a new feature engg
    label=['SalePrice']
    target=['SalePrice']
    df_target=df.loc[:,target]#create view
    
    
    #seller variables: train
    features_c_1=['SaleNbrSellingCntr', 'Brand','StandardBaleDesc']
    features_c_2=['Region','Division','Branch','WAM','Agent']
    features_c=features_c_1+features_c_2
    df_c=df.loc[:,features_c]#create view
    df_c=df_c.fillna('unknown')#replace missing values
    

    df_c=apply_label_encoder(df_c,features_c,'train')#apply label encoder
    #see_correlation(df_c,features_c,'SalePrice',df_target)
    #df_c=meanEncoding(df_c, features_c,'SalePrice',df_target,'train')#apply mean encoder after label encoder, mean encoding displays correlation also
    df_c=apply_impact_encoder(df_c, features_c,'SalePrice',df_target,'train')#apply mean encoder after label encoder, mean encoding displays correlation also
    #check_for_nulls(df_c)
    print(df_c.head(5))
    

    
    #Test set experiment on a new feature engg
    df_c_test= df.loc[0:100]#[-6:] #last 6 rows from raw data
    df_c_test=df_c_test.loc[:,features_c]#create view
    print(df_c_test.head(5))
    df_c_test=df_c_test.fillna('unknown')#replace missing values
    df_c_test=apply_label_encoder(df_c_test,features_c,'test')#apply label encoder
    #see_correlation(df_c,features_c,'SalePrice',df_target)
    #df_c_test=meanEncoding(df_c_test, features_c,'SalePrice',df_target,'test')#apply mean encoder after label encoder, mean encoding displays correlation also
    df_c_test=apply_impact_encoder(df_c_test, features_c,'SalePrice',df_target,'test')#apply mean encoder after label encoder, mean encoding displays correlation also
    #check_for_nulls(df_c)
    print(df_c_test.head(5))



    


Impact coding for SaleNbrSellingCntr
inner


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


inner
Impact coding for Brand
inner
inner
Impact coding for StandardBaleDesc
inner
inner
Impact coding for Region
inner
inner
Impact coding for Division
inner
inner
Impact coding for Branch
inner
inner
Impact coding for WAM
inner
inner
Impact coding for Agent
inner
inner
   SaleNbrSellingCntr        Brand  ...          WAM        Agent
0          982.614976  1312.352941  ...  1171.361615  1182.706972
1          982.615033  1296.334028  ...  1171.361615  1182.706972
2          982.615033  1312.352941  ...  1171.420970  1178.224638
3          982.615033  1312.352941  ...  1171.361615  1178.224638
4          982.614976  1296.334028  ...  1171.361615  1182.706972

[5 rows x 8 columns]


In [ ]:
#Test set experiment on a new feature engg
df_c_test= df.loc[-6:] #last 6 rows from raw data
df_c_test=df_c_test.loc[:,features_c]#create view
print(df_c_test.head(5))
df_c_test=df_c_test.fillna('unknown')#replace missing values
df_c_test=apply_label_encoder(df_c_test,features_c,'test')#apply label encoder
print(df_c_test.head(5))
#see_correlation(df_c,features_c,'SalePrice',df_target)
#df_c_test=meanEncoding(df_c_test, features_c,'SalePrice',df_target,'test')#apply mean encoder after label encoder, mean encoding displays correlation also
df_c_test=apply_impact_encoder(df_c_test, features_c,'SalePrice',df_target,'test')#apply mean encoder after label encoder, mean encoding displays correlation also
#check_for_nulls(df_c)
print(df_c_test.head(5))


  SaleNbrSellingCntr      Brand  ...            WAM              Agent
0                  S  YARRAFORD  ...  NICKY  SYMONS  BP24-BRAD NEWSOME
1                  S  YARRAFORD  ...  NICKY  SYMONS  BP24-BRAD NEWSOME
2                  S  YARRAFORD  ...  NICKY  SYMONS  BP24-BRAD NEWSOME
3                  S  YARRAFORD  ...  NICKY  SYMONS  BP24-BRAD NEWSOME
4                  S  YARRAFORD  ...  NICKY  SYMONS  BP24-BRAD NEWSOME

[5 rows x 8 columns]
label on test data
(632971, 8)
   SaleNbrSellingCntr  Brand  StandardBaleDesc  ...  Branch  WAM  Agent
0                   2  15920               811  ...      91   62     89
1                   2  15920              3989  ...      91   62     89
2                   2  15920              1842  ...      91   62     89
3                   2  15920               811  ...      91   62     89
4                   2  15920               811  ...      91   62     89

[5 rows x 8 columns]
SaleNbrSellingCntr here SaleNbrSellingCntr
0    973.418888
1    949

In [ ]:
a=5
print(a)

5


In [ ]:
#feature set 3 (no mean encoding without not-sold wool, not-sold wool are filtered using filter variable)
def select_engg_features(df_test):#update this function based on selected feature, encoding you want to apply and filter technique
    df=df_test
    #target variable:
    label=['SalePrice']
    target=['SalePrice']
    df_target=df.loc[:,target]#create view


    #weight variables:
    features_w=['NetWeight']#from all the weights only using net weight since others are correlated
    df_w=df.loc[:,features_w]#create view
    df_w=scaleColumns(df_w,features_w)#apply normalization
    #see_correlation(df_w,features_w,'SalePrice',df_target)
    #check_for_nulls(df_w)


    #date variables:
    features_d=['SaleDate']#SaleDate will change to 'Month'
    df_date=df.loc[:,features_d]#create view
    df_date= extract_year_month_day(df_date,'month')#extract month column, this will be added to the dataframe as a new column 'Month'
    df_d=label_encoder(df_date[['Month']],['Month'])#apply label encoder: deal month as categorical or the model would give the most importance to 12
    features_d=['Month']
    #see_correlation(df_d,features_d,'SalePrice',df_target)
    #check_for_nulls(df_d)

    #seller variables:
    features_c_1=['SaleNbrSellingCntr', 'Brand','StandardBaleDesc']
    features_c_2=['Region','Division','Branch','WAM','Agent']
    features_c=features_c_1+features_c_2
    df_c=df.loc[:,features_c]#create view
    df_c.fillna('unknown')#replace missing values
    df_c=apply_label_encoder(df_c,features_c)#apply label encoder
    see_correlation(df_c,features_c,'SalePrice',df_target)
    #df_c=meanEncoding(df_c, features_c,'SalePrice',df_target)#apply mean encoder after label encoder, mean encoding displays correlation also
    #check_for_nulls(df_c)



    #quality variables: 
    features_q_1 = ['Micron','Curvature','VMB','Yield1','FolioProceeds']
    features_q_2=['FDMean','FDCOEfficient','FDComfort']
    features_q_3=['StapleLength','StapleLengthCV','StapleStrength','StapleStrengthLowest25']
    features_q_4=['TheoreticalHauteur','TheoreticalHauteurCV']
    features_q_5=['PositionofBreakTip','PositionofBreakMiddle','PositionofBreakBase']
    features_q=features_q_1+features_q_2+features_q_3+features_q_4+features_q_5
    df_q=df.loc[:,features_q]#create view
    df_q=standardColumns(df_q,features_q)#apply standardization technique
    #see_correlation(df_q,features_q,'SalePrice',df_target)
    #check_for_nulls(df_q)


    #Filter variables(sold or not sold?):
    features_f=['SaleOutcome']
    df_f=df.loc[:,features_f]


    #features you want
    features=features_w+features_d+features_c+features_q+target#you want these features (names only)

    #combine columns
    data=[df_w,df_d,df_c,df_q,df_target]#data = [df1["A"], df2["A"]]
    df_featuere_set = pd.concat(data, axis=1)
    print(df_featuere_set.head(5))

    #filter
    data=[df_w,df_d,df_c,df_q,df_f,df_target]#make data again with filter variable
    df_temp = pd.concat(data, axis=1)#, keys=headers)
    print(df_temp.columns)
    df_temp= df_temp[df_temp['SaleOutcome'] == 0] 
    print(df_temp.head(5))
    df_featuere_set = df_temp.loc[:,features]#dataframe of features without the filter variable


    print(df_featuere_set.head(5))
    return df_featuere_set, features

#feature set 3
df_feature_set_3, feature_names=select_engg_features(df_big) 
df_feature_set_3_train=df_feature_set_3[0:-6]#train data without last 6 data
df_feature_set_3_test=df_feature_set_3[-6:]#last 6 test data FE applied
print(df_feature_set_3.shape)
print(df_feature_set_3_train.shape)
print(df_feature_set_3_test.shape)

#df_feature_set_3_train.to_csv('woolCaret_feature_set_3.csv')

cols to scale ['NetWeight']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


helo see_correlation
SaleNbrSellingCntr  correlation  0.008269741504885807
Brand  correlation  0.00328249202058887
StandardBaleDesc  correlation  -0.15212894381654213
Region  correlation  0.035962264738053064
Division  correlation  0.03682637568384327
Branch  correlation  0.01364686189042153
WAM  correlation  0.023953403099308657
Agent  correlation  -0.03545020848048214
   NetWeight  Month  ...  PositionofBreakBase  SalePrice
0   0.101094      1  ...             1.652904     1250.0
1   0.065628      1  ...             0.418474     1198.0
2   0.086178      1  ...            -0.724517     1208.0
3   0.070103      1  ...            -0.724517     1208.0
4   0.047232      1  ...             0.281315     1411.0

[5 rows x 28 columns]
Index(['NetWeight', 'Month', 'SaleNbrSellingCntr', 'Brand', 'StandardBaleDesc',
       'Region', 'Division', 'Branch', 'WAM', 'Agent', 'Micron', 'Curvature',
       'VMB', 'Yield1', 'FolioProceeds', 'FDMean', 'FDCOEfficient',
       'FDComfort', 'StapleLength', 

In [ ]:
#feature set 2 (with mean encoding and without not-sold wool, not sold wool are filtered)
def select_engg_features(df_test):#update this function based on selected feature, encoding you want to apply and filter technique
    df=df_test
    #target variable:
    label=['SalePrice']
    target=['SalePrice']
    df_target=df.loc[:,target]#create view


    #weight variables:
    features_w=['NetWeight']#from all the weights only using net weight since others are correlated
    df_w=df.loc[:,features_w]#create view
    df_w=scaleColumns(df_w,features_w)#apply normalization
    #see_correlation(df_w,features_w,'SalePrice',df_target)
    #check_for_nulls(df_w)


    #date variables:
    features_d=['SaleDate']#SaleDate will change to 'Month'
    df_date=df.loc[:,features_d]#create view
    df_date= extract_year_month_day(df_date,'month')#extract month column, this will be added to the dataframe as a new column 'Month'
    df_d=label_encoder(df_date[['Month']],['Month'])#apply label encoder: deal month as categorical or the model would give the most importance to 12
    features_d=['Month']
    #see_correlation(df_d,features_d,'SalePrice',df_target)
    #check_for_nulls(df_d)

    #seller variables:
    features_c_1=['SaleNbrSellingCntr', 'Brand','StandardBaleDesc']
    features_c_2=['Region','Division','Branch','WAM','Agent']
    features_c=features_c_1+features_c_2
    df_c=df.loc[:,features_c]#create view
    df_c.fillna('unknown')#replace missing values
    df_c=apply_label_encoder(df_c,features_c)#apply label encoder
    see_correlation(df_c,features_c,'SalePrice',df_target)
    df_c=meanEncoding(df_c, features_c,'SalePrice',df_target)#apply mean encoder after label encoder, mean encoding displays correlation also
    #check_for_nulls(df_c)



    #quality variables: 
    features_q_1 = ['Micron','Curvature','VMB','Yield1','FolioProceeds']
    features_q_2=['FDMean','FDCOEfficient','FDComfort']
    features_q_3=['StapleLength','StapleLengthCV','StapleStrength','StapleStrengthLowest25']
    features_q_4=['TheoreticalHauteur','TheoreticalHauteurCV']
    features_q_5=['PositionofBreakTip','PositionofBreakMiddle','PositionofBreakBase']
    features_q=features_q_1+features_q_2+features_q_3+features_q_4+features_q_5
    df_q=df.loc[:,features_q]#create view
    df_q=standardColumns(df_q,features_q)#apply standardization technique
    #see_correlation(df_q,features_q,'SalePrice',df_target)
    #check_for_nulls(df_q)


    #Filter variables(sold or not sold?):
    features_f=['SaleOutcome']
    df_f=df.loc[:,features_f]


    #features you want
    features=features_w+features_d+features_c+features_q+target#you want these features (names only)

    #combine columns
    data=[df_w,df_d,df_c,df_q,df_target]#data = [df1["A"], df2["A"]]
    df_featuere_set = pd.concat(data, axis=1)
    print(df_featuere_set.head(5))

    #filter
    data=[df_w,df_d,df_c,df_q,df_f,df_target]#make data again with filter variable
    df_temp = pd.concat(data, axis=1)#, keys=headers)
    print(df_temp.columns)
    df_temp= df_temp[df_temp['SaleOutcome'] == 0] 
    print(df_temp.head(5))
    df_featuere_set = df_temp.loc[:,features]#dataframe of features without the filter variable


    print(df_featuere_set.head(5))
    return df_featuere_set, features

#feature set 2
df_feature_set_2, feature_names=select_engg_features(df_big) 
df_feature_set_2_train=df_feature_set_2[0:-6]#train data without last 6 data
df_feature_set_2_test=df_feature_set_2[-6:]#last 6 test data FE applied
print(df_feature_set_2.shape)
print(df_feature_set_2_train.shape)
print(df_feature_set_2_test.shape)

#df_feature_set_2_train.to_csv('woolCaret_feature_set_2.csv')







cols to scale ['NetWeight']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


helo see_correlation
SaleNbrSellingCntr  correlation  0.008269741504885807
Brand  correlation  0.00328249202058887
StandardBaleDesc  correlation  -0.15212894381654213
Region  correlation  0.035962264738053064
Division  correlation  0.03682637568384327
Branch  correlation  0.01364686189042153
WAM  correlation  0.023953403099308657
Agent  correlation  -0.03545020848048214
SaleNbrSellingCntr 0.011214492663550743
Brand 0.46350765706845665
StandardBaleDesc 0.6893155354626512
Region 0.061001556311784506
Division 0.07996289729990345
Branch 0.15462966344792534
WAM 0.15062311662308325
Agent 0.1949854022573914
   NetWeight  Month  ...  PositionofBreakBase  SalePrice
0   0.101094      1  ...             1.652904     1250.0
1   0.065628      1  ...             0.418474     1198.0
2   0.086178      1  ...            -0.724517     1208.0
3   0.070103      1  ...            -0.724517     1208.0
4   0.047232      1  ...             0.281315     1411.0

[5 rows x 28 columns]
Index(['NetWeight', 'Month'

In [ ]:
feature_names
features=feature_names[0:-1]
target=feature_names[-1:]
print(feature_names)
print(target)

['NetWeight', 'Month', 'SaleNbrSellingCntr', 'Brand', 'StandardBaleDesc', 'Region', 'Division', 'Branch', 'WAM', 'Agent', 'Micron', 'Curvature', 'VMB', 'Yield1', 'FolioProceeds', 'FDMean', 'FDCOEfficient', 'FDComfort', 'StapleLength', 'StapleLengthCV', 'StapleStrength', 'StapleStrengthLowest25', 'TheoreticalHauteur', 'TheoreticalHauteurCV', 'PositionofBreakTip', 'PositionofBreakMiddle', 'PositionofBreakBase', 'SalePrice']
['SalePrice']
